In [110]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [147]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [148]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


# Visualizations

In [149]:
import os
import pandas as pd

from virny.utils.custom_initializers import create_models_metrics_dct_from_database_df

from source.utils.db_functions import read_model_metric_dfs_from_db
from source.custom_classes.experiments_composer import ExperimentsComposer
from source.custom_classes.experiments_visualizer import ExperimentsVisualizer

## Initialize Configs

In [150]:
EXPERIMENT_NAME = 'mult_repair_levels'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
DATASET_NAME = 'Folktables_NY_2018_Employment'
# EXPERIMENT_SESSION_UUID = '0b71b2a8-030f-44df-a027-5304af77e2ba'
EXPERIMENT_SESSION_UUID = 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5'
SENSITIVE_ATTRS = ['SEX', 'RAC1P', 'SEX&RAC1P']
MODEL_NAMES = ['RandomForestClassifier']

In [151]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

## Group Metrics Composition

In [152]:
model_metric_dfs = read_model_metric_dfs_from_db(collection_obj, EXPERIMENT_SESSION_UUID)
models_metrics_dct = create_models_metrics_dct_from_database_df(model_metric_dfs)
client.close()

In [153]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].shape

(1980, 34)

In [154]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].head(20)

,Metric,Bootstrap_Model_Seed,Model_Name,Model_Params,Run_Number,Dataset_Name,Num_Estimators,Tag,Record_Create_Date_Time,Session_Uuid,...,SEX&RAC1P_priv,SEX&RAC1P_priv_correct,SEX&RAC1P_priv_incorrect,SEX_dis,SEX_dis_correct,SEX_dis_incorrect,SEX_priv,SEX_priv_correct,SEX_priv_incorrect,overall
0,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 19:21:19.905,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.804070,1.0,0.0,0.776435,1.0,0.0,0.810328,1.0,0.0,0.7935
1,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 22:03:22.818,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.803488,1.0,0.0,0.776435,1.0,0.0,0.809335,1.0,0.0,0.7930
2,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-06 10:37:34.818,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.802907,1.0,0.0,0.775428,1.0,0.0,0.810328,1.0,0.0,0.7930
3,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 21:14:12.662,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.804651,1.0,0.0,0.778449,1.0,0.0,0.810328,1.0,0.0,0.7945
4,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 17:54:49.958,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.808140,1.0,0.0,0.779456,1.0,0.0,0.815293,1.0,0.0,0.7975
5,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-06 15:32:26.983,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.802326,1.0,0.0,0.776435,1.0,0.0,0.808342,1.0,0.0,0.7925
6,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-06 16:55:10.051,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.801163,1.0,0.0,0.774421,1.0,0.0,0.808342,1.0,0.0,0.7915
7,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-06 18:38:32.902,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.801744,1.0,0.0,0.775428,1.0,0.0,0.808342,1.0,0.0,0.7920
8,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 18:32:15.652,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.804070,1.0,0.0,0.775428,1.0,0.0,0.811321,1.0,0.0,0.7935
9,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 19:58:41.636,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.805233,1.0,0.0,0.778449,1.0,0.0,0.810328,1.0,0.0,0.7945


In [155]:
exp_composer = ExperimentsComposer(models_metrics_dct, SENSITIVE_ATTRS)
exp_subgroup_metrics_dct = exp_composer.create_exp_subgroup_metrics_dct()

In [156]:
exp_group_metrics_dct = exp_composer.compose_group_metrics(exp_subgroup_metrics_dct)

In [157]:
exp_subgroup_metrics_dct['RandomForestClassifier']['Exp_iter_1'][0.1].head()

,Metric,Bootstrap_Model_Seed,Model_Name,Model_Params,Run_Number,Dataset_Name,Num_Estimators,Tag,Record_Create_Date_Time,Session_Uuid,...,SEX&RAC1P_priv,SEX&RAC1P_priv_correct,SEX&RAC1P_priv_incorrect,SEX_dis,SEX_dis_correct,SEX_dis_incorrect,SEX_priv,SEX_priv_correct,SEX_priv_incorrect,overall
8,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 18:32:15.652,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.804070,1.000000,0.000000,0.775428,1.000000,0.000000,0.811321,1.000000,0.000000,0.793500
118,Aleatoric_Uncertainty,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 18:32:15.652,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.706701,0.681078,0.811856,0.744918,0.714315,0.850588,0.693961,0.671783,0.789327,0.719261
228,F1,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 18:32:15.652,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.837277,1.000000,0.000000,0.810213,1.000000,0.000000,0.845779,1.000000,0.000000,0.828417
338,FNR,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 18:32:15.652,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.123357,0.000000,1.000000,0.151515,0.000000,1.000000,0.113946,0.000000,1.000000,0.132289
448,FPR,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,200,OK,2023-07-05 18:32:15.652,e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5,...,0.294118,0.000000,1.000000,0.319444,0.000000,1.000000,0.293556,0.000000,1.000000,0.306698


In [158]:
exp_group_metrics_dct['RandomForestClassifier']['Exp_iter_10'][0.1].head()

,Metric,SEX,RAC1P,SEX&RAC1P,Model_Name,Experiment_Iteration,Intervention_Param
0,Equalized_Odds_TPR,0.007639,-0.014816,-0.031254,RandomForestClassifier,Exp_iter_10,0.1
1,Equalized_Odds_FPR,0.033733,0.033712,0.043482,RandomForestClassifier,Exp_iter_10,0.1
2,Equalized_Odds_FNR,-0.007639,0.014816,0.031254,RandomForestClassifier,Exp_iter_10,0.1
3,Disparate_Impact,1.088542,1.005978,1.026397,RandomForestClassifier,Exp_iter_10,0.1
4,Statistical_Parity_Difference,0.092232,0.006501,0.028644,RandomForestClassifier,Exp_iter_10,0.1


## Metrics Visualization and Reporting

In [159]:
visualizer = ExperimentsVisualizer(exp_subgroup_metrics_dct=exp_subgroup_metrics_dct,
                                   exp_group_metrics_dct=exp_group_metrics_dct,
                                   dataset_name=DATASET_NAME,
                                   model_names=MODEL_NAMES,
                                   sensitive_attrs=SENSITIVE_ATTRS)

In [160]:
visualizer.melted_all_group_metrics_per_model_dct[MODEL_NAMES[0]].head(20)

,Metric,Model_Name,Experiment_Iteration,Intervention_Param,Group,Metric_Value
0,Equalized_Odds_TPR,RandomForestClassifier,Exp_iter_1,0.2,SEX,-0.039270
1,Equalized_Odds_FPR,RandomForestClassifier,Exp_iter_1,0.2,SEX,0.018800
2,Equalized_Odds_FNR,RandomForestClassifier,Exp_iter_1,0.2,SEX,0.039270
3,Disparate_Impact,RandomForestClassifier,Exp_iter_1,0.2,SEX,0.993049
4,Statistical_Parity_Difference,RandomForestClassifier,Exp_iter_1,0.2,SEX,-0.007649
5,Accuracy_Parity,RandomForestClassifier,Exp_iter_1,0.2,SEX,-0.033893
6,Label_Stability_Ratio,RandomForestClassifier,Exp_iter_1,0.2,SEX,0.981951
7,IQR_Parity,RandomForestClassifier,Exp_iter_1,0.2,SEX,0.003225
8,Std_Parity,RandomForestClassifier,Exp_iter_1,0.2,SEX,0.002194
9,Std_Ratio,RandomForestClassifier,Exp_iter_1,0.2,SEX,1.043413


### Line band plots for subgroup metrics

In [161]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Accuracy', 'F1', 'FNR', 'Jitter', 'Label_Stability', 'FPR'])

alt.VConcatChart(...)

In [162]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Aleatoric_Uncertainty', 'Overall_Uncertainty'])

alt.VConcatChart(...)

In [163]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Aleatoric_Uncertainty', 'Overall_Uncertainty'],
                                                  subgroup='SEX_dis_correct')

alt.VConcatChart(...)

In [164]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Aleatoric_Uncertainty', 'Overall_Uncertainty'],
                                                  subgroup='SEX_dis_incorrect')

alt.VConcatChart(...)

### Line band plots for group metrics

In [165]:
visualizer.create_group_metrics_line_band_plot(group='SEX',
                                               model_name=MODEL_NAMES[0],
                                               group_metrics=['Equalized_Odds_FPR', 'Equalized_Odds_FNR', 'Label_Stability_Ratio', 'Jitter_Parity'])

alt.VConcatChart(...)